<a href="https://colab.research.google.com/github/noriakihanya/MTF_arrange/blob/main/MTF_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Numpyの設定

!pip uninstall -y numpy
!pip install numpy==1.19.4

print("※処理の完了後、「RESTART RUNTIME」を実行すること")

In [ ]:
#@title ライブラリのインストール

import gdown
gdown.download('https://drive.google.com/u/0/uc?id=1VZkoiVOonffpJWxZah-AdQkxaTFIzZ6q&export=download',quiet=False)

print('Installing dependencies...')
!git clone https://github.com/tensorflow/tensor2tensor.git
%cd /content/tensor2tensor
!pip install -e.

%cd /content
!apt-get update -qq && apt-get install -qq libfluidsynth1 build-essential libasound2-dev libjack-dev
!pip install -q 'tensorflow-datasets < 4.0.0'
!pip install -qU google-cloud magenta pyfluidsynth
!apt install ffmpeg
!apt install fluidsynth
!pip install midi2audio
!pip install pydub
!pip install basic-pitch
!pip install spleeter

!git clone https://github.com/noriakihanya/MTF_arrange.git
import shutil
shutil.copy("/content/MTF_arrange/files/datagen_beam.py","/usr/local/lib/python3.7/dist-packages/magenta/models/socre2perf")
shutil.copy("/content/MTF_arrange/files/gym_utils.py","/content/tensor2tensor/tensor2tensor/rl")
shutil.copy("/content/MTF_arrange/files/t2t_model.py","/content/tensor2tensor/tensor2tensor/utils")
shutil.copy("/content/MTF_arrange/files/transformer.py","/content/tensor2tensor/tensor2tensor/models")

print("※処理の完了後、「RESTART RUNTIME」を実行すること")

In [ ]:
#@title ライブラリのインポート
print('Importing libraries...')

import numpy as np
import os
import tensorflow.compat.v1 as tf

import tensor2tensor
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.data_generators import text_encoder
from tensor2tensor.utils import decoding
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import registry

from magenta.models.score2perf import *
from magenta.models.score2perf import score2perf
import magenta.music as mm
import note_seq

tf.disable_v2_behavior()

from midi2audio import FluidSynth
from IPython.display import Audio
import os
from midi2audio import FluidSynth
import math
from pydub import AudioSegment
from pydub.playback import play
import shutil
import ffmpeg

print('完了!')

In [2]:
#@title 関数定義の設定

# Upload a MIDI file and convert to NoteSequence.
def upload_midi():
  data = list(files.upload().values())
  if len(data) > 1:
    print('Multiple files uploaded; using only one.')
  return note_seq.midi_to_note_sequence(data[0])

# Decode a list of IDs.
def decode(ids, encoder):
  ids = list(ids)
  if text_encoder.EOS_ID in ids:
    ids = ids[:ids.index(text_encoder.EOS_ID)]
  return encoder.decode(ids)

def get_primer_ns(filename, max_length):
    """
    Convert Midi file to note sequences for priming.
    :param filename: Midi file name.
    :param max_length: Maximum note sequence length for priming in seconds.
    :return:
        Note sequences for priming.
    """
    primer_ns = mm.midi_file_to_note_sequence(filename)

    # Handle sustain pedal in primer.
    primer_ns = mm.apply_sustain_control_changes(primer_ns)

    # Trim to desired number of seconds.
    if primer_ns.total_time > max_length:
        LOGGER.warn(
            'Primer duration %d is longer than max second %d, truncating.'
            % (primer_ns.total_time, max_length))
        primer_ns = mm.extract_subsequence(
            primer_ns, 0, max_length
        )

    # Remove drums from primer if present.
    if any(note.is_drum for note in primer_ns.notes):
        LOGGER.warn('Primer contains drums; they will be removed.')
        notes = [note for note in primer_ns.notes if not note.is_drum]
        del primer_ns.notes[:]
        primer_ns.notes.extend(notes)

    # Set primer instrument and program.
    for note in primer_ns.notes:
        note.instrument = 1
        note.program = 0

    return primer_ns

def get_melody_ns(filename):
    """
    Convert melody Midi file to note sequence.
    :param filename: Midi file name.
    :return:
        Melody note sequences.
    """
    melody_ns = mm.midi_file_to_note_sequence(filename)
    melody_instrument = mm.infer_melody_for_sequence(melody_ns)
    # pylint: disable=no-member
    notes = [note for note in melody_ns.notes if note.instrument == melody_instrument]
    del melody_ns.notes[:]
    melody_ns.notes.extend(
        sorted(notes, key=lambda note: note.start_time)
    )
    for i in range(len(melody_ns.notes) - 1):
        melody_ns.notes[i].end_time = melody_ns.notes[i + 1].start_time

    # pylint: disable=no-member

    return melody_ns

print("完了")

完了


In [ ]:
#@title モデルのロード

model_name = 'transformer'
hparams_set = 'transformer_tpu'
ckpt_path = 'gs://magentadata/models/music_transformer/checkpoints/melody_conditioned_model_16.ckpt'

class MelodyToPianoPerformanceProblem(score2perf.AbsoluteMelody2PerfProblem):
  @property
  def add_eos_symbol(self):
    return True

problem = MelodyToPianoPerformanceProblem()
melody_conditioned_encoders = problem.get_feature_encoders()

# Set up HParams.
hparams = trainer_lib.create_hparams(hparams_set=hparams_set)
trainer_lib.add_problem_hparams(hparams, problem)
hparams.num_hidden_layers = 16
hparams.sampling_method = 'random'

# Set up decoding HParams.
decode_hparams = decoding.decode_hparams()
decode_hparams.alpha = 0.0
decode_hparams.beam_size = 1

# Create Estimator.
run_config = trainer_lib.create_run_config(hparams)
estimator = trainer_lib.create_estimator(model_name, hparams, run_config,decode_hparams=decode_hparams)

# These values will be changed by the following cell.
inputs = []
decode_length = 0

# Create input generator.
def input_generator():
    global inputs
    while True:
        yield {
        'inputs': np.array([[inputs]], dtype=np.int32),
        'targets': np.zeros([1, 0], dtype=np.int32),
        'decode_length': np.array(decode_length, dtype=np.int32)
    }

# Start the Estimator, loading from the specified checkpoint.
input_fn = decoding.make_input_fn_from_generator(input_generator())
melody_conditioned_samples = estimator.predict(input_fn, checkpoint_path=ckpt_path)

# "Burn" one.
_ = next(melody_conditioned_samples)

In [4]:
#@title ワークスペースの作成
import os

input = "/content/01_AudioFile_upload/"
os.makedirs(input, exist_ok=True)

spleeter = "/content/02_separate_data/"
os.makedirs(spleeter, exist_ok=True)

basicpitch = "/content/03_bp_output/"
os.makedirs(basicpitch, exist_ok=True)

mtf_arrange = "/content/04_mtf_arrange/"
os.makedirs(mtf_arrange, exist_ok=True)

In [25]:
#@title オーディオファイルのアップロード（MP4 M4A MP3）
shutil.rmtree(input)
os.mkdir(input)

%cd /content/01_AudioFile_upload
from google.colab import files
files.upload()

#mp4からmp3へ変換
audio_file = [os.path.join(input, file) for file in os.listdir(input) if file != '.ipynb_checkpoints']
i = 0
for files in audio_file:
  files = files.replace(" ","")
  files = files.replace("　","")
  os.rename(audio_file[i], files)

audio_file = [os.path.join(input, file) for file in os.listdir(input) if file != '.ipynb_checkpoints']
if ".mp4" in audio_file[i] or ".m4a" in audio_file[i]:
  audio_file_org = audio_file[i]

  input_audio = input + "input_audio.mp4"
  os.rename(audio_file_org, input_audio)

  # 入力 
  stream = ffmpeg.input(input_audio)
  # 出力 
  stream = ffmpeg.output(stream, "output_audio.mp3") 
  # 実行 
  ffmpeg.run(stream)

  os.rename(input_audio,audio_file_org)

/content/01_AudioFile_upload


Saving 08 My Heart Will Go On.m4a to 08 My Heart Will Go On.m4a


In [26]:
#@title Vocalを抽出し、MIDIへ変換
#vocalの抽出
%cd /content

shutil.rmtree(spleeter)
os.mkdir(spleeter)

mp3_file = [os.path.join(input, file) for file in os.listdir(input) if '.mp3' in file]
input_mp3 = mp3_file[0]

!spleeter \
separate -o $spleeter/ \
$input_mp3

#MIDIへ変換
shutil.rmtree(basicpitch)
os.mkdir(basicpitch)

folderfile = spleeter + os.listdir(spleeter)[0]
spleeter_file = [os.path.join(folderfile, file) for file in os.listdir(folderfile) if file != '.ipynb_checkpoints']
for files in spleeter_file:
  if "vocals.wav" in files:
    vocal_file = files

!basic-pitch \
$basicpitch \
$vocal_file

#MIDIの展開
midi_file = [os.path.join(basicpitch, file) for file in os.listdir(basicpitch) if '.mid' in file]
input_midi = midi_file[0]
melody_ns = note_seq.midi_file_to_note_sequence(input_midi)

SAMPLE_RATE = 16000
SF2_PATH = '/content/Essential Keys-sforzando-v9.6.sf2'

melody_instrument = note_seq.infer_melody_for_sequence(melody_ns)
notes = [note for note in melody_ns.notes if note.instrument == melody_instrument]
del melody_ns.notes[:]
melody_ns.notes.extend(sorted(notes, key=lambda note: note.start_time))
for i in range(len(melody_ns.notes) - 1):
    melody_ns.notes[i].end_time = melody_ns.notes[i + 1].start_time
inputs = melody_conditioned_encoders['inputs'].encode_note_sequence(melody_ns)

note_seq.play_sequence(melody_ns,synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH)
note_seq.plot_sequence(melody_ns)

/content
INFO:spleeter:File /content/02_separate_data/output_audio/vocals.wav written succesfully
INFO:spleeter:File /content/02_separate_data/output_audio/vocals.wav written succesfully
INFO:spleeter:File /content/02_separate_data/output_audio/accompaniment.wav written succesfully
INFO:spleeter:File /content/02_separate_data/output_audio/accompaniment.wav written succesfully

✨✨✨✨✨✨✨✨✨
✨ Basic Pitch  ✨
✨✨✨✨✨✨✨✨✨

Importing Tensorflow (this may take a few seconds)...

Predicting MIDI for /content/02_separate_data/output_audio/vocals.wav...


  Creating midi...
  💅 Saved to /content/03_bp_output/vocals_basic_pitch.mid

✨ Done ✨



/usr/local/lib/python3.7/dist-packages/pretty_midi/pretty_midi.py:295: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  current_instrument = np.zeros(16, dtype=np.int)
/usr/local/lib/python3.7/dist-packages/note_seq/melody_inference.py:319: RuntimeWarning: divide by zero encountered in log
  melody_transition_distribution[pitch_indices, :][:, pitch_indices])
/usr/local/lib/python3.7/dist-packages/note_seq/melody_inference.py:184: RuntimeWarning: divide by zero encountered in log
  mat = durations[:, np.newaxis] * np.log(mat)


In [19]:
#@title MusicTransformerアレンジ

SAMPLE_RATE = 16000
SF2_PATH_MTF = '/content/Essential Keys-sforzando-v9.6.sf2'

# Generate sample events.
decode_length = 4096
sample_ids = next(melody_conditioned_samples)['outputs']

# Decode to NoteSequence.
midi_filename = decode(sample_ids,encoder=melody_conditioned_encoders['targets'])
accompaniment_ns = note_seq.midi_file_to_note_sequence(midi_filename)

# Play and plot.
note_seq.play_sequence(accompaniment_ns,synth=note_seq.fluidsynth, sample_rate=SAMPLE_RATE, sf2_path=SF2_PATH_MTF)
note_seq.plot_sequence(accompaniment_ns)

In [20]:
#@title 作成されたMIDIをMP3に変換してダウンロード
MTF_arr = '/content/04_mtf_arrange/MTF_arrange.mid'
note_seq.sequence_proto_to_midi_file(accompaniment_ns, MTF_arr)

fs = FluidSynth(sound_font=SF2_PATH_MTF)
MTF_mp3 = "/content/04_mtf_arrange/MTF_arrange.mp3"
fs.midi_to_audio(MTF_arr, MTF_mp3)

audio = AudioSegment.from_file(MTF_mp3)
# 音量(%)
volume_percent = 500

# 音量を調整して再生
mtf = audio + (20 * math.log10(volume_percent/100))
mtf.export(MTF_mp3, format='mp3')

from google.colab import files
files.download("/content/04_mtf_arrange/MTF_arrange.mp3")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>